In [1]:
# Define some parameters
factor = 150
data_dir = "./data/dataset"
save_path = "data/datasetBF"
partitions_dir = "partitions.parquet"
splits = ['train', 'val', 'test']

In [2]:
import os
import gc
import random

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import deque
from pyvis.network import Network
from lib.store import Store
from tqdm.notebook import tqdm
import fsspec
from collections import Counter
from typing import Dict
from sklearn.preprocessing import MinMaxScaler
import random


from lib.naming_corrections import (
    FEATURE_COLUMNS_OTHERS,
    FEATURES_NAMES_FROM_NEW_CACHE,
    FEATURES_NAMES_FROM_PRELOADED_CACHE,
    TABLES_COLUMNS_DEFAULT_LEGACY,
    TABLES_V5_2_V4_RENAME_LEGACY,
)

FEATURE_COLUMNS = FEATURES_NAMES_FROM_PRELOADED_CACHE + FEATURE_COLUMNS_OTHERS
TABLES_V5_2_V4_RENAME = TABLES_V5_2_V4_RENAME_LEGACY
TABLES_COLUMNS_DEFAULT = TABLES_COLUMNS_DEFAULT_LEGACY

In [33]:
def bfs_custom(G, source, label_dict, neighbors=None, depth_limit=None, ratio=None, visited=None):
    '''
    Generic breath first seach algorithm
    '''
    # Define the depth of the search
    if depth_limit is None:
        depth_limit = len(G)
        
    # neighbors(source): return a generator of source's neighbors
    queue = deque([(source, depth_limit, neighbors(source))])

    # Placeholder for edges
    edges = []

    # Run the queue
    while queue:
        parent, depth_now, children = queue[0]
        try:
            # Logic for random first seach
            child = next(children)

            # If the child has not been visited before
            if child not in visited:
                # Construct an edge
                edges.append((parent, child))

                # Update visited
                visited.add(child)

                # Increment the ratio
                ratio[label_dict[child]] += 1

                # Incremen the child
                if depth_now > 1:
                    queue.append((child, depth_now - 1, neighbors(child)))
                    
            # Break the ratio of the model
            try:
                if ratio[0]/ratio[1] >= factor:
                    break
            except ZeroDivisionError as e:
                pass

        # If no more nodes in the queue
        except StopIteration:
            queue.popleft()

    # Return the edges
    return edges, ratio, visited

def n_depth_search(G, frontiers, neighbors, label_dict, depth, ratio, visited, edges):
    '''
    Define the depth until the nodes are important
    '''
    # Get the nodes for the source
    new_set = set()
    for front_node in frontiers:
        # Add the front node to the visited
        visited.add(front_node)

        # Maintain the ratio
        ratio[label_dict[front_node]] += 1
        
        # Collect the nodes
        neighbors_ = neighbors(front_node)

        # Loop the neighbors
        for neigh in neighbors_:
            # Updated the visited
            visited.add(neigh)

            # Updates the edges
            edges.append([front_node, neigh])

            # Make the new frontier list
            new_set.add(neigh)

            # Add the ratio to the list
            ratio[label_dict[neigh]] += 1

    return new_set, ratio, visited, edges

def frontier_sampling(frontiers, edges, neighbors, ratio, visited):
    '''
    Performs frontier sampling on the frontier to chose initial nodes
    '''
    # Define the prob of selection 
    degrees = [G.degree(i) for i in frontiers]
    probs = [G.degree(i) / sum(degrees) for i in frontiers]

    # Choose a frontier
    chosen_frontier = np.random.choice(list(frontiers), p=probs, size=1)[0]

    # Get the neighbors of the chosen frontier
    frontier_neighbors = list(neighbors(chosen_frontier))

    # Randomly select a child node
    random_child = random.choice(frontier_neighbors)

    if random_child not in visited:
        # Add the edge
        edges.append([chosen_frontier, random_child])
    
        # Update the ratio
        ratio[label_dict[random_child]] += 1
    
        # Update the frontiers
        frontiers.remove(chosen_frontier)
        visited.add(chosen_frontier)
        frontiers.add(random_child)

    # Return frontiers
    return frontiers, edges, ratio, visited

def frontier_bfs(G, source, label_dict, neighbors=None, depth_limit=None, sort_neighbors=None, factor=100, depth=1):
    '''
    Uses frontiers to start and then uses breath first seach to complete the models
    '''
    # Define the ratio
    ratio = {0: 0, 1: 0}

    # Define the visited nodes
    visited = set()

    # Placeholder for edges
    edges = []

    # Define the frontier nodes
    frontiers = set([source])

    # Stage 1 frontiers
    for _ in range(0, depth):
        frontiers, ratio, visited, edges = n_depth_search(G, frontiers, neighbors,
                                                         label_dict, 1, ratio,
                                                         visited, edges)

    # Stage 2 sampling (fronier sampling)
    depth_val = random.randint(2, 5)
    for _ in range(depth_val):
        frontiers, edges, ratio, visited = frontier_sampling(frontiers, edges, neighbors, ratio, visited)

    assert source not in frontiers, "Error source in frontier after stage 2"

    # Update the visisted
    visited.update(frontiers)

    # Define random probs for the frontiers
    frontier_probs = list(np.random.rand(len(frontiers)))
    frontier_list = list(frontiers)

    # Sort the list based on the probs
    frontiers_with_probs_sorted = sorted([[frontier_list[i], frontier_probs[i]] for i in range(len(frontier_list))],
                                         key=lambda x : x[-1],
                                         reverse=True)
    
    # Loop over the frontiers
    for front_node, _ in frontiers_with_probs_sorted:
        # Run BFS on the frontier to collect its node
        edges_bfs, ratio, visited = bfs_custom(G=G, source=front_node, label_dict=label_dict,
                                                neighbors=neighbors, depth_limit=None,
                                                ratio=ratio, visited=visited)

        # Add to the edges
        edges.extend(edges_bfs)
    
    # Return the edges
    return edges

In [17]:
def generic_bfs_edges(G, source, label_dict, neighbors=None, depth_limit=None, sort_neighbors=None, factor=100):
    '''
    Simple BFS
    '''
    if callable(sort_neighbors):
        _neighbors = neighbors
        neighbors = lambda node: iter(sort_neighbors(_neighbors(node)))
    
    ratio = {0: 0, 1: 0} 
    visited = {source}
    if depth_limit is None:
        depth_limit = len(G)

    queue = deque([(source, depth_limit, neighbors(source))])
    ratio[label_dict[source]] += 1
    
    edges = []
    while queue:
        parent, depth_now, children = queue[0]
        try:
            # Logic for random first seach
            child = next(children)
            if child not in visited:
                edges.append((parent, child))
                visited.add(child)
                ratio[label_dict[child]] += 1
                if depth_now > 1:
                    queue.append((child, depth_now - 1, neighbors(child)))
            try:
                if ratio[0]/ratio[1] >= factor:
                    return edges
            except ZeroDivisionError as e:
                pass
        except StopIteration:
            queue.popleft()
            
    return edges

In [18]:
def generic_mhrw_edges(G, source, label_dict, neighbors=None, depth_limit=None, sort_neighbors=None, factor=100):
    """
    Iterate over edges in a MHR search.
    """

    # Define the ratio 
    ratio = {0: 0, 1: 0}
    visited = {source}

    # Define the depth limit
    if depth_limit is None:
        depth_limit = len(G)

    # No need for a queue
    node_list = set([source])
    parent_node = source
    parent_neighbors = list(neighbors(source))
    node_list.update(parent_neighbors)

    # Find the degree of the parent node
    degree_parent = G.degree(source)

    # Maintain the ratio
    ratio[label_dict[source]] += 1

    # Placeholder for collected edges
    edges = []
    set_all_added_nodes = set()

    # Define the logic for MHRW
    idx = 0
    while (ratio[0]/ratio[1]) < factor:
        # Check if node list is not empty
        if len(node_list) > 0:
            # Get the next child
            child = node_list.pop()
    
            # Define the probability
            p = round(random.uniform(0, 1), 4)
    
            # Add nodes and update visited
            if child not in visited and label_dict[child] != 1:
                # Neighbors of child
                child_neighbors = neighbors(child)
    
                # Collect the degree of the child
                degree_child = G.degree(child)
    
                # Check if probability constraint is satisfied
                if ((p <= min(1, degree_parent / degree_child)) and (child in list(neighbors(parent_node)))) or (idx == 0):
                    # Update the edges and other list
                    edges.append((parent_node, child))

                    # Update the global list
                    set_all_added_nodes.update((parent_node, child))

                    # Uodate the visited and ratio
                    visited.add(child)
                    ratio[label_dict[child]] += 1
                    
                    # Replace the data
                    parent_node = child
                    degree_parent = degree_child
                    node_list.clear()
                    node_list.update(child_neighbors)
                    idx += 1
        else:
            # Update the nodelist with random nodes
            node_list.update(set(random.sample(list(set(G.nodes()) - set_all_added_nodes), 3)))
            
            # Get the new parent
            parent_node = node_list.pop()
            
            # Refresh the node list
            degree_parent = G.degree(parent_node)
            node_list.clear()
            node_list.update(list(neighbors(parent_node)))

    # Return the data
    return edges

In [19]:
def generic_frontier_edges(G, source, label_dict, neighbors=None, depth_limit=None,
                           sort_neighbors=None, factor=100, num_frontiers=20):
    """
    Iterate over edges in a MHR search.
    """

    # Define the ratio 
    ratio = {0: 0, 1: 0}
    visited = {source}
    break_condition = False

    # Define the depth limit
    if depth_limit is None:
        depth_limit = len(G)

    # Define the frontier nodes
    frontier_nodes = set()
    neighbors_s_node = set(neighbors(source))

    # Find the reachable frontier nodes
    reachable_nodes = list(nx.single_source_shortest_path_length(G, source).keys())
    reachable_nodes.remove(source)

    # Add some random nodes just in case
    temp_ = list(np.random.choice(list(G.nodes), size=num_frontiers))
    reachable_nodes.extend(temp_)
    reachable_nodes = [i for i in reachable_nodes if i in label_dict][:num_frontiers]

    # Update the frontier nodes
    frontier_nodes.update(reachable_nodes)
    
    # Maintain the ratio
    ratio[label_dict[source]] += 1

    # Placeholder for collected edges
    edges = [(source, i) for i in frontier_nodes]

    # Add the seed nodes to the mix
    for node in frontier_nodes:
        ratio[label_dict[node]] += 1

    # Define the patience
    patience = 0

    # Loop and collect edgeneric_frontier_edges
    while (ratio[0]/ratio[1]) < factor:
        # Select a new node with some probability
        degrees = [G.degree(i) for i in frontier_nodes]
        list_probs = [G.degree(i) / sum(degrees) for i in frontier_nodes]
        
        # Get the next child
        chosen_frontier = np.random.choice(list(frontier_nodes), p=list_probs, size=1)[0]

        # Neighbors of child
        frontier_neighbors = list(neighbors(chosen_frontier))

        # Randomly chose one of them
        random_child = random.choice(frontier_neighbors)

        # Add nodes and update visited
        if random_child not in visited and label_dict[random_child] != 1:
            # Update the edges and other list
            edges.append((chosen_frontier, random_child))
            ratio[label_dict[random_child]] += 1

            # Replace u by v in the node list
            frontier_nodes.remove(chosen_frontier)
            frontier_nodes.add(random_child)

            # Reset patience
            patience = 0
        else:
            # Increment the patience
            patience += 1

            if break_condition:
                break
            
            # Edge case for unreacheable nodes
            if patience == 50:
                frontier_nodes = visited - set([source])
                break_condition = True
            
    # Return the data
    return edges

In [20]:
def load_local_data_store(data_dir:str) -> Store:   
    # build the store
    store = Store(
        base_dir=data_dir,
        protocol='file'
    )
    return store

datastore = load_local_data_store(data_dir)
df_p = pd.read_parquet(
        datastore.open_file(partitions_dir)
    ).reset_index(drop=True).reset_index()

In [21]:
counters = {}
graph_data = {}
labelled = {}
others = {}
for sp, A in df_p.groupby('split'):
    graph_data[sp] = {
        x: None
        for x in sorted(A['index'])
    }
    labelled[sp] = {
        x: None
        for x in sorted(A['index'])
    }
    counters[sp] = Counter()

In [22]:
def fit_scaler(graph_data: Dict):
    print("Fitting Scaler...")
    scaler = MinMaxScaler()
    for p in tqdm(graph_data):
        df_f = pd.read_parquet(f"./data/dataset/cache/features/features_{p}.parquet")
        X = df_f[FEATURE_COLUMNS].fillna(value=0.0).values
        scaler.partial_fit(X)
        del df_f, X
        gc.collect()
    return scaler

scaler = fit_scaler(graph_data['train'])

Fitting Scaler...


  0%|          | 0/377 [00:00<?, ?it/s]

In [23]:
def post_process_graph(result, network, labels, features, idx, name_sampling):
    # Convert the FS seach to dataframe from merging and stuff
    df_result = pd.DataFrame(result, columns=['from', 'to'])
    
    # Undirected to directed
    directed1 = network.merge(df_result, how='inner', left_on=['from', 'to'], right_on=['from', 'to'])
    directed2 = network.merge(df_result, how='inner', left_on=['from', 'to'], right_on=['to', 'from'])[['from_x', 'to_x', 'partition']].rename(columns={"from_x": "from", "to_x": "to", 'partition': 'partition'})
    samples = pd.concat([directed1, directed2], axis=0)
    edges_list = samples[['from', 'to']].values

    # Get the unique graph edges
    df_node = pd.DataFrame(set(edges_list.reshape(-1)), columns=['node'])

    # Process the features and labels
    sample_labels = labels.merge(df_node, how='inner').sort_values(by=['node'])
    sample_features = features.merge(sample_labels, how='inner', on='txid').sort_values(by=['node']).drop(['node'], axis=1)
    mapping = dict(zip(sample_labels.node.values, range(len(sample_labels))))
    samples[['from', 'to']] = samples[['from', 'to']].replace(mapping)
    sample_labels.node = [i for i in range(len(sample_labels))]
    sample_features[FEATURE_COLUMNS] = scaler.transform(sample_features[FEATURE_COLUMNS].values)

    # Save the data
    samples.to_parquet(f'./sampling_comparison/{name_sampling}_edges_{idx}.parquet')
    sample_labels.to_parquet(f'./sampling_comparison/{name_sampling}_labels_{idx}.parquet')
    sample_features.to_parquet(f'./sampling_comparison/{name_sampling}_features_{idx}.parquet')

In [32]:
# New parition file
p_dict = {'index': [], 'split': []}

# Placeholder
idx = 0

# Loop over splits
for sp in splits:
    # Loop over split data
    bar = tqdm(graph_data[sp], total=len(graph_data[sp]))
    for p in bar:
        
        # Load the required files
        network = pd.read_parquet(f'./{data_dir}/cache/edges/edges_{p}.parquet')
        labels = pd.read_parquet(f'./{data_dir}/labels/labels_{p}.parquet')
        features = pd.read_parquet(f'./{data_dir}/cache/features/features_{p}.parquet')
    
        # Convert to network x graph
        G = nx.from_pandas_edgelist(network, 'from', 'to')
        successors = G.neighbors # can retrieve all neighbors of a particular node with []
        
        # Replace all label 2 as label 0
        labels.loc[labels['label'] == 2, 'label'] = 0
        label_dict = dict(zip(labels.node, labels.label))
    
        # Construct graph from each positive node
        for _, pos_node in enumerate(labels[labels['label']==1].node.values):
            try: # the node in the label parquet may not exist in the edge parquet
                result_bfs = generic_bfs_edges(G, pos_node, label_dict, successors, factor=factor)
                result_bf = frontier_bfs(G, pos_node, label_dict, successors, factor=factor)
                result_mhrw = generic_mhrw_edges(G, pos_node, label_dict, successors, factor=factor)
                result_fs = generic_frontier_edges(G, pos_node, label_dict, successors, factor=factor)
            except Exception as e:
                continue

            # Save the samples together
            post_process_graph(result_bfs, network, labels, features, idx, "bfs")
            post_process_graph(result_bf, network, labels, features, idx, "bf")
            post_process_graph(result_mhrw, network, labels, features, idx, "mhrw")
            post_process_graph(result_fs, network, labels, features, idx, "fs")

            # Increment the idx
            bar.set_description(f"File saved : {idx}")
            idx += 1

  0%|          | 0/377 [00:00<?, ?it/s]

[[7490, 0.390408064784192], [7487, 0.15388339440543775]]
[[3937, 0.12257089530959497]]
[[24806, 0.5117702517678953], [14188, 0.45293267867024], [93978, 0.0021971178125413937]]
[[62538, 0.809696187013152], [61456, 0.2711873162973757]]
[[95803, 0.19319611212477028], [89158, 0.17130706679801355]]
[[42595, 0.8208524411942988], [43750, 0.5588573338655894]]
[[45698, 0.9696252287600801], [80999, 0.38463761897670634]]
[[8842, 0.9111472998104647], [12447, 0.26250234562139774]]
[[82561, 0.8936573228057618], [82581, 0.18201330946386685]]
[[68993, 0.7211278976933982], [57144, 0.07941631567692331]]
[[84471, 0.18041576945383742]]
[[20415, 0.7200954138395627], [15032, 0.4402247153037092], [3276, 0.3208041570046397], [39525, 0.27404642966844783], [17727, 0.10632137138137021], [10861, 0.09287175015632021], [27497, 0.024728069654555318]]
[[48759, 0.9722293125318281], [76161, 0.6796119200618356]]
[[89298, 0.7018313946711479]]
[[44199, 0.4098546807612493], [41593, 0.2252678937190118]]
[[27996, 0.741683748

KeyboardInterrupt: 